In [14]:
import numpy as np
from machine_learning.xgb_def import XGBOptimizer
from machine_learning.reglog_def import REGLOGOptimizer
import pandas as pd
from opt_funct.data_fuct import functions
from sklearn.model_selection import train_test_split
import time
import optuna
from sklearn.metrics import f1_score, classification_report
from machine_learning.combination1 import Combined_xgb_logreg
from machine_learning.SVM_defopt import SVMOptimizer
from itertools import combinations
from opt_funct.data_fuct import Operation

ImportError: cannot import name 'Operation' from 'opt_funct.data_fuct' (/Users/rixon/Desktop/ITESO/11/Trading/Proyecto3/003_ML/opt_funct/data_fuct.py)

In [2]:
df = functions.clean_ds(pd.read_csv('./data/aapl_5m_train.csv'))
df

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN,NaN,NaN,131.448898,NaN,False,True
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,132.809997,NaN,NaN,131.119995,NaN,False,True
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,131.889999,132.809997,NaN,130.875000,NaN,False,True
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,132.059997,131.889999,132.809997,130.509902,NaN,False,True
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,132.250000,132.059997,131.889999,130.889999,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,129.039993,129.020004,128.910003,NaN,56.509324,False,False
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,129.054992,129.039993,129.020004,NaN,60.700323,False,False
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,129.380004,129.054992,129.039993,NaN,64.854367,False,False
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,129.764404,129.380004,129.054992,NaN,66.620919,False,False


**Entrenamos nuestros modelos para compra y venta**

In [3]:
# Buy
close_data = df[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']].dropna()
x = close_data.drop(columns=['Y_BUY'])
y = close_data[['Y_BUY']]
x_train_buy, x_test_buy, y_train_buy, y_test_buy = train_test_split(x, y, test_size=0.3, shuffle=False)

close_data2 = df[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_SELL']].dropna()
x = close_data2.drop(columns=['Y_SELL'])
y = close_data2[['Y_SELL']]
x_train_sell, x_test_sell, y_train_sell, y_test_sell = train_test_split(x, y, test_size=0.3, shuffle=False)

In [4]:
# buy
xgboost_optimizer = XGBOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
xgboost_best_params_buy, best_model_xgboost_buy = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
reg_log_best_params_buy, best_model_reg_log_buy = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train_buy, y_train_buy, x_test_buy, y_test_buy)
SVC_best_params_buy, best_model_SVC_buy = SVC_optimizer.svm_optuna()

# sell
xgboost_optimizer = XGBOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
xgboost_best_params_sell, best_model_xgboost_sell = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
reg_log_best_params_sell, best_model_reg_log_sell = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train_sell, y_train_sell, x_test_sell, y_test_sell)
SVC_best_params_sell, best_model_SVC_sell = SVC_optimizer.svm_optuna()

[I 2024-03-09 14:29:56,397] A new study created in memory with name: no-name-e4a56584-8c1c-4e95-915d-e44edfd31319
/Users/rixon/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [14:29:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "gamma", "max_depth", "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-03-09 14:29:56,484] Trial 0 finished with value: 0.678879670360265 and parameters: {'n_estimators': 77, 'max_depth': 25, 'max_leaves': 17, 'learning_rate': 1.0671256127156539, 'booster': 'gblinear', 'gamma': 22.879353919375724, 'reg_alpha': 2.7134997442366418, 'reg_lambda': 5.960750444828362}. Best is trial 0 with value: 0.678879670360265.
[I 2024-03-09 14:29:58,310] Trial 1 finished with value: 0.678879670360265 and parameters: {'n_estimators': 65, 'max_depth': 73, 'max_leaves': 9, 'learning_rate': 1.8162780462317663, 'booster': 'dart', 'gamma': 49.09964004602691, 'reg_alpha': 4.184664793027661, 'reg_lambda

F1: 0.678879670360265
Best hyperparameters: {'n_estimators': 77, 'max_depth': 25, 'max_leaves': 17, 'learning_rate': 1.0671256127156539, 'booster': 'gblinear', 'gamma': 22.879353919375724, 'reg_alpha': 2.7134997442366418, 'reg_lambda': 5.960750444828362}
Execution time: 2.07717338403066 minutes


[I 2024-03-09 14:32:03,071] Trial 0 finished with value: 0.5817598600277029 and parameters: {'C': 6.424064279499859, 'fit_intercept': False, 'l1_ratio': 0.22907517239140496}. Best is trial 0 with value: 0.5817598600277029.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-09 14:32:05,103] Trial 1 finished with value: 0.5818155325659332 and parameters: {'C': 5.0761293441663105, 'fit_intercept': False, 'l1_ratio': 0.5679963942507399}. Best is trial 1 with value: 0.5818155325659332.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2

F1 Score: 0.6135597789459495
Best hyperparameters: {'C': 0.001885996402740113, 'fit_intercept': True, 'l1_ratio': 0.7604411067373734}
Execution time: 2.733373781045278 minutes


/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-09 14:34:47,724] Trial 0 finished with value: 0.43819813221021786 and parameters: {'C': 1.922164142049238, 'kernel': 'linear', 'degree': 5, 'gamma': 4.0}. Best is trial 0 with value: 0.43819813221021786.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-09 14:34:48,533] Trial 1 finished with va

...

F1: 0.58906910181273
Best hyperparameters: {'n_estimators': 78, 'max_depth': 99, 'max_leaves': 25, 'learning_rate': 1.32742878948571, 'booster': 'dart', 'gamma': 12.834018142558051, 'reg_alpha': 9.041478800518927, 'reg_lambda': 3.541012818093309}
Execution time: 6.37533305088679 minutes


[I 2024-03-09 14:53:42,370] A new study created in memory with name: no-name-2f702feb-67ce-439e-a5f9-91ba7b2213f6
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-09 14:53:44,230] Trial 0 finished with value: 0.3221957040572792 and parameters: {'C': 1.8437821649810997, 'fit_intercept': True, 'l1_ratio': 0.26053910571037586}. Best is trial 0 with value: 0.3221957040572792.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-09 14:56:11,318] Trial 1 finished with value: 0.33896724559972025 and parameters: {'C': 7.736170058891

F1 Score: 0.3398205337373267
Best hyperparameters: {'C': 7.766532665566769, 'fit_intercept': False, 'l1_ratio': 0.3127111274177694}
Execution time: 12.196536564826966 minutes


/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-09 15:08:14,239] Trial 0 finished with value: 0.0 and parameters: {'C': 3.609549252316383, 'kernel': 'sigmoid', 'degree': 4, 'gamma': 'auto'}. Best is trial 0 with value: 0.0.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-09 15:08:15,053] Trial 1 finished with value: 0.5919633225458468 and 

F1 Score: 0.6510887486497242
Best hyperparameters: {'C': 6.477799669185736, 'kernel': 'sigmoid', 'degree': 5, 'gamma': 4.0}
Execution time: 15.800265880425771 minutes


In [9]:
# Indicadores Buy
close_data['xg_boost_buy'] = best_model_xgboost_buy.predict(x)
close_data['reg_log_buy'] = best_model_reg_log_buy.predict(x)
close_data['SVC_buy'] = best_model_SVC_buy.predict(x)
# Indicadores Sell
close_data = close_data.drop(columns=['Y_BUY','RSI','X_t-1','X_t-2','X_t-3'])
close_data['xg_boost_sell'] = best_model_xgboost_sell.predict(x)
close_data['reg_log_sell'] = best_model_reg_log_sell.predict(x)
close_data['SVC_sell'] = best_model_SVC_sell.predict(x)
# Convertimos a boleanos
close_data['xg_boost_buy'] = np.where(close_data['xg_boost_buy']==1, True, False)
close_data['xg_boost_sell'] = np.where(close_data['xg_boost_sell']==1, True, False)
close_data

,Close,xg_boost_buy,reg_log_buy,SVC_buy,xg_boost_sell,reg_log_sell,SVC_sell
27,129.413299,True,False,True,True,True,False
28,129.148498,True,False,True,True,True,False
29,128.820007,True,False,True,True,True,False
30,128.580001,True,False,True,True,True,False
31,128.351699,True,False,True,True,True,False
...,...,...,...,...,...,...,...
39574,129.054992,True,True,True,True,False,False
39575,129.380004,True,True,True,False,False,False
39576,129.764404,True,True,True,False,False,False
39577,129.949996,True,True,True,False,False,False


**Mostramos nuestras combinaciones posibles de compra y venta**

In [14]:
indicadores_compra = ['xg_boost_buy', 'reg_log_buy' , 'SVC_buy']
indicadores_venta = ['xg_boost_sell', 'reg_log_sell' , 'SVC_sell']

combinaciones_indicadores_compra = []
combinaciones_indicadores_venta = []

for i in range(1, len(indicadores_compra) + 1):
    combinaciones_indicadores_compra.extend(combinations(indicadores_compra, i))

for i in range(1, len(indicadores_venta) + 1):
    combinaciones_indicadores_venta.extend(combinations(indicadores_venta, i))
combinaciones_indicadores_compra, combinaciones_indicadores_venta


([('xg_boost_buy',),
  ('reg_log_buy',),
  ('SVC_buy',),
  ('xg_boost_buy', 'reg_log_buy'),
  ('xg_boost_buy', 'SVC_buy'),
  ('reg_log_buy', 'SVC_buy'),
  ('xg_boost_buy', 'reg_log_buy', 'SVC_buy')],
 [('xg_boost_sell',),
  ('reg_log_sell',),
  ('SVC_sell',),
  ('xg_boost_sell', 'reg_log_sell'),
  ('xg_boost_sell', 'SVC_sell'),
  ('reg_log_sell', 'SVC_sell'),
  ('xg_boost_sell', 'reg_log_sell', 'SVC_sell')])

In [13]:
cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]

In [12]:
xgboost_best_params_buy

{'n_estimators': 77,
 'max_depth': 25,
 'max_leaves': 17,
 'learning_rate': 1.0671256127156539,
 'booster': 'gblinear',
 'gamma': 22.879353919375724,
 'reg_alpha': 2.7134997442366418,
 'reg_lambda': 5.960750444828362}

In [15]:
class Operation:
    def __init__(self, operation_type, bought_at, timestamp, n_shares,
                 stop_loss,take_profit):
        self.operation_type = operation_type
        self.bought_at = bought_at
        self.timestamp = timestamp
        self.n_shares = n_shares
        self.sold_at = None
        self.stop_loss = stop_loss
        self.take_profit = take_profit

In [ ]:
def optimize(trial, strat):
    stop_loss_long = trial.suggest_float('stop_loss_long', 0.01, 0.3)
    take_profit_long = trial.suggest_float('take_profit_long', 0.01, 0.3)
    stop_loss_short = trial.suggest_float('stop_loss_short', 0.01, 0.3)
    take_profit_short = trial.suggest_float('take_profit_short', 0.01, 0.3)
    n_shares = trial.suggest_int('n_shares', 10, 100)
   

    strategy_params = {}

    buy_signals = pd.DataFrame()
    sell_signals = pd.DataFrame()

    if "xg_boost_buy" in strat:
        strategy_params["xg_boost_buy"] = {xgboost_best_params_buy}
        
        rsi = RSI(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["RSI"]["window"], strategy_params["RSI"]["upper_treshold"], strategy_params["RSI"]["lower_treshold"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        rsi_buy_signals, rsi_sell_signals, rsi_strategy_value = rsi.run_strategy()
        buy_signals["RSI"] = rsi_buy_signals
        sell_signals["RSI"] = rsi_sell_signals
    if "EWMA" in strat:
        strategy_params["EWMA"] = {
        "long_window" : trial.suggest_int('ewma_long_window', 50, 100),
        "short_window" : trial.suggest_int('ewma_short_window', 5, 49)
        }
        ewma = EWMA(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["EWMA"]["long_window"], strategy_params["EWMA"]["short_window"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        ewma_buy_signals, ewma_sell_signals, ewma_strategy_value = ewma.run_strategy()
        buy_signals["EWMA"] = ewma_buy_signals
        sell_signals["EWMA"] = ewma_sell_signals

    if "SMA" in strat:
        strategy_params["SMA"] = {
        "long_window" : trial.suggest_int('sma_long_window', 20, 50),
        "short_window" : trial.suggest_int('sma_short_window', 2, 19)
        }
        sma = SMA(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["SMA"]["long_window"], strategy_params["SMA"]["short_window"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        sma_buy_signals, sma_sell_signals, sma_strategy_value = sma.run_strategy()
        buy_signals["SMA"] = sma_buy_signals
        sell_signals["SMA"] = sma_sell_signals
    
    if "ADX" in strat:
        strategy_params["ADX"] = {
        "window" : trial.suggest_int('adx_window', 10, 50),
        "threshold" : trial.suggest_int('adx_threshold', 15, 30)
        }
        adx = ADX(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["ADX"]["window"], strategy_params["ADX"]["threshold"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        adx_buy_signals, adx_sell_signals, adx_strategy_value = adx.run_strategy()
        buy_signals["ADX"] = adx_buy_signals
        sell_signals["ADX"] = adx_sell_signals

    if "STO" in strat:
        strategy_params["STO"] = {
        "window" : trial.suggest_int('sto_window', 10, 50),
        "upper_threshold" : trial.suggest_int('sto_upper_threshold', 50, 95),
        "lower_threshold" : trial.suggest_int('sto_lower_threshold', 5, 49)
        }
        sto = STO(df_5m, cash, active_operations, com, strategy_value, n_shares, strategy_params["STO"]["window"], strategy_params["STO"]["upper_threshold"], strategy_params["STO"]["lower_threshold"], stop_loss_long, take_profit_long, stop_loss_short, take_profit_short)
        sto_buy_signals, sto_sell_signals, sto_strategy_value = sto.run_strategy()
        buy_signals["STO"] = sto_buy_signals
        sell_signals["STO"] = sto_sell_signals

    
    # Backtesting
    for i, row in buy_signals.iterrows():
        # If all indicators want to buy...
        if sum(buy_signals.iloc[i,:]) == len(buy_signals.columns):
            buy_signal = True
        else:
            buy_signal = False
        buy_signals.loc[i, "General"] = buy_signal

    for i, row in sell_signals.iterrows():
        # If all indicators want to sell...
        if sum(sell_signals.iloc[i,:]) == len(sell_signals.columns):
            sell_signal = True
        else:
            sell_signal = False
        sell_signals.loc[i, "General"] = sell_signal

    
    return strategy_value[-1]

best_value_overall = float('-inf')
best_params_overall = {}

for strat in combinaciones_indicadores:
    pruner = optuna.pruners.MedianPruner()
    study = optuna.create_study(direction='maximize',sampler=optuna.samplers.TPESampler(), pruner=pruner)
    study.optimize(lambda trial: optimize(trial, strat), n_trials=50, n_jobs=-1)

    # Get the best parameters and their corresponding value
    best_params = study.best_params
    best_value = study.best_value

    if best_value > best_value_overall:
        best_value_overall = best_value
        best_params_overall = best_params

    print("Best parameters for combination:", strat)
    print("Best value:", best_value)
    print("Best parameters:", best_params)
    print()

print("Overall best value:", best_value_overall)
print("Overall best parameters:", best_params_overall)